In [1]:
from oct2py import Oct2Py

import os
import pandas as pd
import csv

In [2]:

import sys
sys.path.append('allCodes')
# from F_LabH2 import F_LabH2_return_constraints
from F_DifEvo import F_DifEvo_LH2_return_constraints_fitness, F_DifEvo_LH2_return_constraints


In [3]:
V_S = 7.0                   # service speed [kn]

In [4]:
seed = 1

Fitness and Evaluation

In [5]:
# replicated dif evo
new_fitness = False

# modified fitness dif evo
# new_fitness = True

In [7]:
# run the Octave script to get the power brake for the params
def run_dif_evo(V_S, seed):
    filename_allRun = 'allRunSaved_' + str(seed) + '.csv'
    if new_fitness:
        D, Z, AEdAO, PdD, P_B = F_DifEvo_LH2_return_constraints_fitness(V_S, filename_allRun)
    else:
        D, Z, AEdAO, PdD, P_B = F_DifEvo_LH2_return_constraints(V_S, filename_allRun)
    return [D, Z, AEdAO, PdD, P_B]

In [ ]:
# run the Octave script to get the power brake for the params
def run_dif_evo(V_S, seed):
    P_B, t075dD,tmin075dD, tal07R,cavLim, Vtip,Vtipmax = [-1, -1, -1, -1, -1, -1, -1]
    output_filename = 'df_' + str(seed) + '.csv'
    with Oct2Py() as octave:
        octave.warning ("off", "Octave:data-file-in-path");
        octave.addpath('./allCodesOctave');
        octave.eval('pkg load statistics')
        # replicated
        if dif_evo_version == 0:
            D, Z, AEdAO, PdD, P_B = octave.F_DifEvo_LH2_return_constraints(V_S, output_filename, nout=5)
        # modified fitness
        elif dif_evo_version == 1:
            D, Z, AEdAO, PdD, P_B = octave.F_DifEvo_LH2_return_constraints_fitness(V_S, output_filename, nout=5)
    return [D, Z, AEdAO, PdD, P_B]

In [8]:
run_dif_evo(V_S, seed)

seed 2631
1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
2
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
3
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
4
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
5
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
6
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
7
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
8
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
9
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270


[0.8, 5.0, 0.657403204870551, 0.684639927558343, 81.49118044340864]

## Make CSV

In [13]:
solver_name = 'DE_mod' if new_fitness else 'DE'

filename_allRun = 'allRunSaved_' + str(seed) + '.csv'


In [15]:
def get_best_fit_number(x):
    for i in range(len(x)):
        x.iloc[i] = x.iloc[i][len('best fit at iteration '):]
    return x

col_names_original = ['D', 'Z', 'AEdAO', 'PdD', 'P_B', 'n', 'etaO', 'etaR', 't075dD','tmin075dD', 'tal07R','cavLim', 'Vtip','Vtipmax', 'fitness', 'iteration', 'population_i']
col_names = ['D', 'AEdAO', 'PdD', 'Z', 'P_B', 'n', 'fitness', 't075dD','tmin075dD', 'tal07R','cavLim', 'Vtip','Vtipmax', 'penalty', 'valid']

filename = 'df_' + str(seed) + '.csv'

df = pd.read_csv(filename_allRun, header=None, skiprows=1, names=col_names_original)

df = df[['D', 'AEdAO', 'PdD', 'Z', 'P_B', 'n', 'fitness', 't075dD','tmin075dD', 'tal07R','cavLim', 'Vtip','Vtipmax']]

# -- Calculate validity --

# insert the other collumns
df.insert(df.shape[1], 'penalty', None)

df.insert(df.shape[1], 'valid', True)

# create temporary cols
df['temp_cav'] = (df['tal07R'] - df['cavLim'])/df['cavLim']
df['temp_cav'] = df['temp_cav'].clip(lower=0)
df['temp_spd'] = (df['Vtip'] - df['Vtipmax'])/df['Vtipmax']
df['temp_spd'] = df['temp_spd'].clip(lower=0)
df['temp_str'] = (df['tmin075dD'] - df['t075dD'])/df['tmin075dD']
df['temp_str'] = df['temp_str'].clip(lower=0)
# calculate
df['penalty'] = df['temp_cav'] + df['temp_spd'] + df['temp_str']
df['valid'] = (df['penalty'] == 0)
# drop temporary cols
df = df.drop(['temp_cav', 'temp_spd', 'temp_str'], axis=1)


# ajust the iteration counter with fitness
df['D'] = df['D'].astype(str)
df.loc[df['D'].str.contains('best'), 'fitness'] = df.loc[df['D'].str.contains('best'), 'Z']
df.loc[df['D'].str.contains('best'), 'Z'] = None

df.loc[df['D'].str.contains('best'), 'AEdAO'] = get_best_fit_number(df.loc[df['D'].str.contains('best'), 'D'])
df.loc[df['D'].str.contains('best'), 'D'] = 'fitness at iteration'

# -- add best value to the end --
best_fit_to_find = df.loc[len(df)-1]['fitness'] # best fitness at last interation
result_row = df[(df['P_B'] == best_fit_to_find) & (df['D'] != 'fitness at iteration')] # find row with best fitness
df.loc[len(df)] = {'D': 'Best Solution'} # add text to the bottom
df.loc[len(df)] = result_row.iloc[0] # add row with values in the end

# save file
filename_out = str(seed)+'.csv'
df.to_csv(filename_out, index=False, header=True)

print('saved:', filename_out)
df.tail(10)

saved: 1.csv


/tmp/ipykernel_12881/473570090.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['D'].str.contains('best'), 'AEdAO'] = get_best_fit_number(df.loc[df['D'].str.contains('best'), 'D'])


,D,AEdAO,PdD,Z,P_B,n,fitness,t075dD,tmin075dD,tal07R,cavLim,Vtip,Vtipmax,penalty,valid
952,0.8,0.800859,0.871377,5.0,83.967245,603.876727,83.967245,0.01225,0.009667,0.218605,0.227185,25.295129,39.0,0.000000,1.0
953,0.8,0.623878,0.724568,6.0,0.000000,672.502353,0.000000,0.01100,0.009208,0.218744,0.201622,28.169713,39.0,0.084921,0.0
954,0.8,1.05,0.707715,6.0,85.033456,691.210428,85.033456,0.01100,0.009259,0.122621,0.195388,28.953355,39.0,0.000000,1.0
955,0.8,0.729247,0.902891,5.0,0.000000,591.915726,0.000000,0.01225,0.009664,0.251746,0.231863,24.794108,39.0,0.085751,0.0
956,0.8,0.429192,0.657738,6.0,0.000000,715.936777,0.000000,0.01100,0.009207,0.276442,0.187472,29.989090,39.0,0.474582,0.0
957,0.8,0.879807,0.946563,7.0,84.688138,554.155070,84.688138,0.00975,0.009007,0.240110,0.247840,23.212393,39.0,0.000000,1.0
958,0.8,0.567333,0.527484,3.0,84.583033,886.359671,84.583033,0.01475,0.010563,0.132856,0.141304,37.127747,39.0,0.000000,1.0
959,fitness at iteration,30,NaN,NaN,NaN,NaN,84.583033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
960,Best Solution,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
961,0.8,0.567333,0.527484,3.0,84.583033,886.359671,84.583033,0.01475,0.010563,0.132856,0.141304,37.127747,39.0,0.000000,True


# best result file

create file

In [16]:
dir_name = '.'

# -- functions to save a file and create a dir --
def create_file(text):
    filename = dir_name+'/' + text +'.csv'
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        header = ["D = propeller diameter [m]",
                  "AEdAO = expanded area ratio",
                  "PdD = pitch ratio",
                  "Z = propeller's number of blades",
                  "P_B = power brake",
                  "n = Propeller angular speed [rpm]",
                  "fitness"]
        writer.writerow(header)
    return filename

def append_to_file(filename, row):
    with open(filename, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)
        
def append_to_file_order(filename, D='', AEdAO='', PdD='', Z='', P_B='', n='', fitness='', i=''):
    row = [D, AEdAO, PdD, Z, P_B, n, fitness, i]
    with open(filename, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)

def save_best_result(D, Z, AEdAO, PdD, fitness, seed, solver_name, P_B='', n='', history=''):
    # create the csv file with the headers
    fitness = -fitness
    Z = int(Z)
    filename = create_file('best_results_' + str(seed) + '_' + solver_name)
    append_to_file_order(filename, D=D, AEdAO=AEdAO, PdD=PdD, Z=Z, P_B=P_B, n=P_B, fitness=fitness)
    append_to_file(filename, ['history'])
    append_to_file(filename, history)
    print('saved', filename)

In [20]:
D       = float(result_row.iloc[0]['D'])
Z       = int(result_row.iloc[0]['Z'])
AEdAO   = float(result_row.iloc[0]['AEdAO'])
PdD     = float(result_row.iloc[0]['PdD'])
fitness = float(result_row.iloc[0]['fitness'])
P_B = float(result_row.iloc[0]['P_B'])
n = float(result_row.iloc[0]['n'])

# history
filename = 'df_' + str(seed) + '.csv'
history = []
with open(filename, 'r', newline='') as file:
    reader = csv.reader(file, delimiter=',')
    for row in reader:
        if 'best' in row[0]:
            # iteration = row[0].split(' ')[-1]
            fit = -float(row[1])
            history.append(str(fit))

save_best_result(D, Z, AEdAO, PdD, fitness, seed, 'original', P_B=P_B, n=n, history=history)

saved ./best_results_1_original.csv
